#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com) - (very minor modifications by Barney Ales)  [Click here for Max Woolf's orignal Colab Notebook.](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce)

*Last updated: 12 Feb 2020*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
import pickle
from datetime import datetime
from google.colab import files
import json


TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

## GPU

Colaboratory uses either Nvidia T4 GPU, Nvidia P100 GPU, or an Nvidia K80 GPU. The T4 and P100 are slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [0]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('\nYour runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Sat Apr 25 22:26:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [0]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 933Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 153Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 1.14Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:01, 256Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 538Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 211Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 227Mit/s]                                                       


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [0]:
file_name = "Sputnik_body_alltext.txt" #this is a ~2 million word text file that contains 1 month of Sputnik news articles

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [0]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess, 
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:14<00:00, 14.13s/it]


dataset has 2523546 tokens
Training...
[10 | 30.14] loss=2.93 avg=2.93
[20 | 52.89] loss=3.05 avg=2.99
[30 | 76.15] loss=2.88 avg=2.95
[40 | 99.96] loss=3.17 avg=3.01
[50 | 123.59] loss=2.91 avg=2.99
[60 | 146.97] loss=2.86 avg=2.96
[70 | 170.34] loss=2.72 avg=2.93
[80 | 193.86] loss=3.04 avg=2.94
[90 | 217.38] loss=2.93 avg=2.94
[100 | 240.87] loss=2.72 avg=2.92
[110 | 264.36] loss=2.86 avg=2.91
[120 | 287.84] loss=2.84 avg=2.91
[130 | 311.33] loss=3.01 avg=2.91
[140 | 334.83] loss=3.01 avg=2.92
[150 | 358.32] loss=2.63 avg=2.90
[160 | 381.81] loss=2.74 avg=2.89
[170 | 405.29] loss=2.85 avg=2.89
[180 | 428.77] loss=2.78 avg=2.88
[190 | 452.26] loss=2.66 avg=2.87
[200 | 475.75] loss=2.92 avg=2.87
======== SAMPLE 1 ========
 this is not acceptable" because he "has a lot of friends … [he] had a lot of times" with Baskauskas during the campaign, the media outlet reported.

In a statement to Reuters, Baskauskas said: "The President is concerned and is responding to all calls. It is necessa

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

The below code creates a list of 150 randomly generated strings of token length 800 and saves them as a json file.  Note: token length roughly equates to word count, but since GPT-2 uses Byte Pair Encoding to create the tokens in its vocabulary, so tokens are usually parts of words.  For more info on tokenization read [the illustrated gpt-2](http://jalammar.github.io/illustrated-gpt2/).  Using a GPU powered Colab Notebook it takes roughly 13 seconds to generate each text string.  Since the generation process is lengthy and resource intensive text strings are best created in batches and then combined together, as Colab can run out of memory, idle timeout, or deactive your session, which all will crash your generation run.

In [0]:
%cd /content
gpt_text_stored = []
for n in range(0, 150):
    print("Round: ", str(n))
    gpt_text_stored.append(gpt2.generate(sess, run_name='run1', length=800, return_as_list=True)[0])

%cd /content/drive/'My Drive'/'Colab Notebooks'
with open('gpt_sputnik_text_data_6.json', 'w', encoding='utf-8') as f:
   json.dump(gpt_text_stored, f, ensure_ascii=False, indent=4)
%ls

/content
Round:  0
Round:  1
Round:  2
Round:  3
Round:  4
Round:  5
Round:  6
Round:  7
Round:  8
Round:  9
Round:  10
Round:  11
Round:  12
Round:  13
Round:  14
Round:  15
Round:  16
Round:  17
Round:  18
Round:  19
Round:  20
Round:  21
Round:  22
Round:  23
Round:  24
Round:  25
Round:  26
Round:  27
Round:  28
Round:  29
Round:  30
Round:  31
Round:  32
Round:  33
Round:  34
Round:  35
Round:  36
Round:  37
Round:  38
Round:  39
Round:  40
Round:  41
Round:  42
Round:  43
Round:  44
Round:  45
Round:  46
Round:  47
Round:  48
Round:  49
Round:  50
Round:  51
Round:  52
Round:  53
Round:  54
Round:  55
Round:  56
Round:  57
Round:  58
Round:  59
Round:  60
Round:  61
Round:  62
Round:  63
Round:  64
Round:  65
Round:  66
Round:  67
Round:  68
Round:  69
Round:  70
Round:  71
Round:  72
Round:  73
Round:  74
Round:  75
Round:  76
Round:  77
Round:  78
Round:  79
Round:  80
Round:  81
Round:  82
Round:  83
Round:  84
Round:  85
Round:  86
Round:  87
Round:  88
Round:  89
Round:  90


If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.